In [ ]:
#%pip install -r requirements.txt

In [ ]:
spark_version = "3.5"
scala_version = "2.12"
iceberg_version = "1.7.0"

from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, rand, floor, expr

catalog_name = "iceberg"
warehouse_path = "./icehouse"

spark = SparkSession.builder \
    .appName("iceberg_timetravel_stuff") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version}") \
    .getOrCreate()

In [ ]:
namespace = "stock_stuff"
spark.sql(f"create namespace {namespace}")

spark.sql(f"""create or replace table {catalog_name}.{namespace}.tickers
        using iceberg
        as
        select 'MSFT' as ticker_symbol, 23.99 as price
        union all
        select 'SNOW', 34.99 as price
        """)


In [3]:
spark.sql(f"select * from {catalog_name}.{namespace}.tickers").show()

+-------------+-----+
|ticker_symbol|price|
+-------------+-----+
|         MSFT|23.99|
|         SNOW|34.99|
+-------------+-----+



In [4]:
df = spark.sql("""
    select 'MSFT' as ticker_symbol, 21.45 as price
""")
df.createOrReplaceTempView("data")

In [ ]:
spark.sql(f"""
    MERGE INTO {catalog_name}.{namespace}.tickers as tgt
        USING data as src
            on tgt.ticker_symbol = src.ticker_symbol
        WHEN MATCHED THEN UPDATE set tgt.price = src.price
        WHEN NOT MATCHED THEN INSERT *
""")

DataFrame[]

In [8]:
spark.sql(f"select * from {catalog_name}.{namespace}.tickers").show()

+-------------+-----+
|ticker_symbol|price|
+-------------+-----+
|         MSFT|21.45|
|         SNOW|34.99|
+-------------+-----+



In [ ]:
#spark.sql(f"select * from {catalog_name}.{namespace}.tickers.snapshots").show()
spark.sql(f"select * from {catalog_name}.{namespace}.tickers TIMESTAMP AS OF '2025-04-28T06:03:00'").show()
spark.sql(f"select * from {catalog_name}.{namespace}.tickers TIMESTAMP AS OF '2025-04-28T06:04:00'").show()

+-------------+-----+
|ticker_symbol|price|
+-------------+-----+
|         MSFT|21.45|
|         SNOW|34.99|
+-------------+-----+



In [ ]:
spark.sql(f"select * from {catalog_name}.{namespace}.tickers.changelog").show()
"""
    might need to write a query that joins the history table back to itself to see full audit
    https://iceberg.apache.org/docs/1.8.0/spark-queries/#inspecting-tables

    if that doesn't work, then a python script to loop through the changes might be needed...
"""


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `iceberg`.`stock_stuff`.`tickers`.`changelog` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [iceberg, stock_stuff, tickers, changelog], [], false
